### Load the packages

In [ ]:
library(rstan)
library(bayesplot)
library(ggplot2)
library(doParallel)
library(devtools)
Sys.setenv(PATH = paste("C:/rtools40/mingw64/bin/", Sys.getenv("PATH"), sep=";"))
Sys.setenv(BINPREF = "C:/rtools40/mingw64/bin/")

In [3]:
rstan_options(auto_write=TRUE)
options(mc.cores=parallel::detectCores())
registerDoParallel(4)
getDoParWorkers()

[1] 4

### Define a function to prepare the parameters and data for further Bayesian analysis

In [4]:
getpara<-function(filename=NULL,names.mk=NULL){
          data.mk<-read.csv(filename)
          names.study<-unique(fixlen.str(data.mk$Study))
          hash.study<-1:length(names.study)
          names(hash.study)<-names.study
          hash.mk<-1:length(names.mk)
          names(hash.mk)<-names.mk
          para<-data.frame(study.names=fixlen.str(data.mk$Study),
          Study=array(hash.study[as.character(fixlen.str(data.mk$Study))]),
                      TP=round(data.mk$Case*data.mk$Sensitivity),
                      TN=round(data.mk$Control*data.mk$Specificity),
                      Dis=data.mk$Case,
                      NDis=data.mk$Control,
                      Test=array(hash.mk[as.character(data.mk$Marker)]))
    return(list(para=para,data=data.mk))
  }


### Define a function to get the upper and lower credible bound (97.5%)

In [5]:
get.ub.95<-function(x){
            y=sort(x)
            len=length(x)
            return(y[round(len*0.975)])
  }

get.lb.95<-function(x){
            y=sort(x)
            len=length(x)
            return(y[round(len*0.025)])
  }

### Define a function to get the results from Network Meta-analysis

In [6]:
get.nma<-function(data.ls=NULL,mk.names=NULL,n.study=1,conf=95){
	        data.nma<-data.ls$data.nma
	        func.ub<-NULL
        	func.lb<-NULL
	        if(conf==95){
		                    func.ub<-get.ub.95
		                    func.lb<-get.lb.95
	        } else if(conf==90){
		                           func.ub<-get.ub.90
		                           func.lb<-get.lb.90
	        }
	        data.nma.muse<-data.nma[paste('MU[',1,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.musp<-data.nma[paste('MU[',2,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.rrse<-data.nma[paste('RR[',1,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.rrsp<-data.nma[paste('RR[',2,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.orse<-data.nma[paste('OR[',1,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.orsp<-data.nma[paste('OR[',2,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.dor<-data.nma[paste('DOR[',1:length(data.ls$mk),']',sep='')]
            data.nma.sindex<-data.nma[paste('S[',1:length(data.ls$mk),']',sep='')]
            data.nma.sindex_se<-data.nma[paste('S_se[',1:length(data.ls$mk),']',sep='')]
            data.nma.sindex_sp<-data.nma[paste('S_sp[',1:length(data.ls$mk),']',sep='')]
	        mu.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.musp,mean))),
	                            ub_sp=array(unlist(lapply(data.nma.musp,func.ub))),
	                            lb_sp=array(unlist(lapply(data.nma.musp,func.lb))),
	                            mean_se=array(unlist(lapply(data.nma.muse,mean))),
	                            ub_se=array(unlist(lapply(data.nma.muse,func.ub))),
	                            lb_se=array(unlist(lapply(data.nma.muse,func.lb))))
	        rr.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.rrsp,mean))),
	                            ub_sp=array(unlist(lapply(data.nma.rrsp,func.ub))),
	                            lb_sp=array(unlist(lapply(data.nma.rrsp,func.lb))),
	                            mean_se=array(unlist(lapply(data.nma.rrse,mean))),
	                            ub_se=array(unlist(lapply(data.nma.rrse,func.ub))),
	                            lb_se=array(unlist(lapply(data.nma.rrse,func.lb))))
	        or.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.orsp,mean))),
	                            ub_sp=array(unlist(lapply(data.nma.orsp,func.ub))),
	                            lb_sp=array(unlist(lapply(data.nma.orsp,func.lb))),
	                            mean_se=array(unlist(lapply(data.nma.orse,mean))),
	                            ub_se=array(unlist(lapply(data.nma.orse,func.ub))),
	                            lb_se=array(unlist(lapply(data.nma.orse,func.lb))))
	        dor.info<-data.frame(mean=array(unlist(lapply(data.nma.dor,mean))),
	                             ub=array(unlist(lapply(data.nma.dor,func.ub))),
	                             lb=array(unlist(lapply(data.nma.dor,func.lb))))
            sindex.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex,mean))),
	                                ub=array(unlist(lapply(data.nma.sindex,func.ub))),
	                                lb=array(unlist(lapply(data.nma.sindex,func.lb))))
            sindex_se.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_se,mean))),
	                                   ub=array(unlist(lapply(data.nma.sindex_se,func.ub))),
	                                   lb=array(unlist(lapply(data.nma.sindex_se,func.lb))))
            sindex_sp.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_sp,mean))),
	                                   ub=array(unlist(lapply(data.nma.sindex_sp,func.ub))),
	                                   lb=array(unlist(lapply(data.nma.sindex_sp,func.lb))))
	        results.nma<-list(marker=mk.names[mk.names %in% data.ls$mk],
	                          mu=mu.info,
	                          rr=rr.info,
	                          or=or.info,
	                          dor=dor.info,
                              sindex=sindex.info,
                              sindex_se=sindex_se.info,
                              sindex_sp=sindex_sp.info)
	   return(results.nma)
}

### Define a function to extract results from analysis

In [8]:
fixlen.str<-function(string.arr=NULL){
                                     	 max.len<-max(nchar(as.character(string.arr)))
	                                     rule<-paste('% -',as.character(max.len),'s',sep='')
	                                     string.arr.fix<-sprintf(rule,string.arr)
	                                     return(string.arr.fix)
}
fit.extract<-function(fit.data=NULL,filename.save=NULL){
	                                                       results.sum<-summary(fit.data)[[1]]
	                                                       results.simdata<-fit.data@sim$samples[[1]]
	                                                       save(results.sum,results.simdata,file=filename.save)
	                                                       return(results.simdata)
}

##########################################################
###  Network Meta-analysis (stan code)
##########################################################

In [9]:
stan.code.nma<-'
data{
     int N;  //number of comparison??? - 121
     int Nt; //number of test - 10
     int Ns; //number of study - 72
     int TP[N];
     int Dis[N];  //diseased
     int TN[N];
     int NDis[N]; //non-diseased
     int Study[N];
     int Test[N];
}
parameters{
           matrix[2, Nt] logitmu;
           vector[Ns] nu[2];
           matrix[Ns, Nt] delta[2];
           vector<lower=0>[Nt] tau[2]; //*
           vector<lower=0>[2] sigmab; 
           real<lower=-1, upper=1> rho;
}
transformed parameters{
                       matrix[Ns, 2] p_i[Nt];
                       matrix[2, Nt] MU;
                       matrix[2, Nt] RR;
                       matrix[2, Nt] OR;
                       vector[Nt] DOR;
                       vector[Nt] S;
                       vector[Nt] S_se;
                       vector[Nt] S_sp;
                       matrix[Nt, Nt] A;
                       matrix[Nt, Nt] B;
                       matrix[Nt, Nt] C;
                       matrix[Nt, Nt] A_se;
                       matrix[Nt, Nt] B_se;
                       matrix[Nt, Nt] C_se;
                       matrix[Nt, Nt] A_sp;
                       matrix[Nt, Nt] B_sp;
                       matrix[Nt, Nt] C_sp;
                       vector<lower=0>[Nt] tausq[2];
                       vector<lower=0>[2] sigmabsq;
                       matrix[Nt, Nt] sigmasq[2];
                       matrix[Nt, Nt] rhow[2];

    for (i in 1:Ns){
        for (j in 1:2){
            for (k in 1:Nt)
                p_i[k][i,j] = inv_logit(logitmu[j,k] +  nu[j][i] + delta[j][i,k]);
        }
    }
 
    for (j in 1:2){
        for (k in 1:Nt){
                         MU[j,k] = mean(col(p_i[k], j));
        }
        tausq[j] = (tau[j]).*(tau[j]);
    }

    for (j in 1:2){
        for (k in 1:Nt){
                         RR[j, k] = MU[j, k]/MU[j, 1]; 
                         OR[j, k] = (MU[j, k]*(1 - MU[j, 1]))/(MU[j, 1]*(1 - MU[j, k]));
         }
    }

    for (l in 1:Nt){
                     DOR[l] = (MU[1, l]*MU[2, l])/((1 - MU[1, l])*(1 - MU[2, l]));

        for(m in 1:Nt){
                        A[l, m] = if_else((MU[1, l] > MU[1, m]) && (MU[2, l] > MU[2, m]), 1, 0);
                        B[l, m] = if_else((MU[1, l] < MU[1, m]) && (MU[2, l] < MU[2, m]), 1, 0);
                        C[l, m] = if_else((MU[1, l] == MU[1, m]) && (MU[2, l] == MU[2, m]), 1, 0);

                        A_se[l, m] = if_else((MU[1, l] > MU[1, m]), 1, 0);
                        B_se[l, m] = if_else((MU[1, l] < MU[1, m]), 1, 0);
                        C_se[l, m] = if_else((MU[1, l] == MU[1, m]), 1, 0);

                        A_sp[l, m] = if_else((MU[2, l] > MU[2, m]), 1, 0);
                        B_sp[l, m] = if_else((MU[2, l] < MU[2, m]), 1, 0);
                        C_sp[l, m] = if_else((MU[2, l] == MU[2, m]), 1, 0);
        }

        S[l] = (2*sum(row(A, l)) + sum(row(C, l)))/(2*sum(row(B, l)) + sum(row(C, l)));
        S_se[l] = (2*sum(row(A_se, l)) + sum(row(C_se, l)))/(2*sum(row(B_se, l)) + sum(row(C_se, l)));
        S_sp[l] = (2*sum(row(A_sp, l)) + sum(row(C_sp, l)))/(2*sum(row(B_sp, l)) + sum(row(C_sp, l)));
    }
    
    sigmabsq = (sigmab).*(sigmab);

    for (j in 1:2){
        for (k in 1:Nt){
            for (l in 1:Nt){
                             sigmasq[j][k,l] = (sigmabsq[j] + tausq[j][k])*((sigmabsq[j] + tausq[j][l]));
                             rhow[j][k,l] = sigmabsq[j]/sqrt(sigmasq[j][k,l]);
            }
        }
    }

}
model{
	   //Priors
       for (j in 1:2){
                       logitmu[j] ~ normal(0, 5);
		               tau[j] ~ cauchy(0, 2.5);
       }

         sigmab ~ cauchy(0, 2.5);
	     rho ~ uniform(-1, 1);
         nu[2] ~ normal(0, sigmab[2]);
      
       for (i in 1:Ns){
                       nu[1][i] ~ normal((sigmab[1]/sigmab[2])*rho*nu[2][i], sqrt(sigmabsq[1]*(1 - (rho*rho))));
          for (j in 1:2){
              for (k in 1:Nt)
                              delta[j][i,k] ~ normal(0, tau[j][k]);
        }
    }

    for (n in 1:N){
                    TP[n] ~ binomial(Dis[n], p_i[Test[n]][Study[n], 1]);
                    TN[n] ~ binomial(NDis[n], p_i[Test[n]][Study[n], 2]);
    }

}
generated quantities{
    
    vector[2*N] loglik;

    for (n in 1:N)
                   loglik[n] = binomial_lpmf(TN[n]| NDis[n], p_i[Test[n]][Study[n], 1]);

    for (n in (N+1):(2*N))
                   loglik[n] = binomial_lpmf(TN[n-N]| NDis[n-N], p_i[Test[n-N]][Study[n-N], 2]);

}
'

### Lists of Biomarkers in network meta-analysis 

In [10]:
mk.names=c('qSOFA',
           'PCT',
           'presepsin',
           'CRP',
           'CD64',
           'IL-6',
           'sTREM-1',
           'LBP',
           'SIRS',
           'SOFA')

### Data set to used

In [ ]:
filename<-'Sepsis_nma_data.csv'

### Parameters lists

In [ ]:
para.ls<-getpara(filename=filename,names.mk=mk.names)
para<-para.ls$para
data<-para.ls$data

### Prepare data for Network Meta-analysis

In [ ]:
para.as.ls.nma<-list(N=nrow(para),
                     Nt=length(unique(para$Test)),
                     Ns=length(unique(para$Study)),
                     TP=para$TP,
                     Dis=para$Dis,
                     TN=para$TN,
                     NDis=para$NDis,
                     Study=as.numeric(para$Study),
                     Test=para$Test)

### Compiling model

In [11]:
stan.model.nma<-stan(model_code=stan.code.nma,
                     data=para.as.ls.nma, 
                     iter=1, 
                     warmup=0, 
                     chains=2)

Warning message:
"There were 2 divergent transitions after warmup. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"There were 1 chains where the estimated Bayesian Fraction of Missing Information was low. See
http://mc-stan.org/misc/warnings.html#bfmi-low"
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"


### Running model

In [12]:
start_time <- Sys.time()
fit.nma<-stan(fit=stan.model.nma,
              data=para.as.ls.nma,
              iter=20000,
              chains=4,
              cores=4,
              control=list(adapt_delta=0.99,
                           stepsize=0.01,
                           max_treedepth=17))
end_time <- Sys.time()
end_time - start_time

Warning message:
"There were 3 divergent transitions after warmup. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
http://mc-stan.org/misc/warnings.html#bfmi-low"
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"
Warning message:
"The largest R-hat is NA, indicating chains have not mixed.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#r-hat"
Warning message:
"Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess"
Warning message:
"Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more 

Time difference of 12.61385 hours

### Save the results as .RData and .csv

In [13]:
filename.simdata.save<-'fit_nma_simdata_qSOFAasreference.RData'
filename.results.save<-'out_nma_sepsis3_qSOFAasreference.csv'
results.nma<-fit.extract(fit.data=fit.nma,filename.save=filename.simdata.save)
results.nma.95<-get.nma(data.ls=list(mk=mk.names[unique(para$Test)],
                                         data.nma=results.nma),mk.names=mk.names,
                            conf=95)
write.csv(results.nma.95,filename.results.save)

### Print the results of pooled sensitivity and specificity from Network Meta-analysis

In [14]:
rownames(results.nma.95$mu)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$mu

,mean_sp,ub_sp,lb_sp,mean_se,ub_se,lb_se
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
qSOFA,0.9350399,0.9851064,0.7152199,0.3441670,0.5782082,0.1758317
PCT,0.7479259,0.7720325,0.7205966,0.7355990,0.7612861,0.7070737
presepsin,0.7328942,0.7937598,0.6529221,0.7983973,0.8487443,0.7378142
CRP,0.7022493,0.7442045,0.6570001,0.6793854,0.7225907,0.6322632
CD64,0.7825432,0.9089204,0.5728038,0.7869876,0.9242837,0.5304800
IL-6,0.7445243,0.8390475,0.6297946,0.7220959,0.7947150,0.6369029
sTREM-1,0.7599295,0.9364470,0.3599408,0.5090747,0.8029378,0.2181196
LBP,0.5980572,0.8360774,0.2872839,0.6146363,0.8459895,0.2868398
SIRS,0.5298767,0.6749732,0.3716796,0.7698001,0.8709361,0.6088603


### Print the results of pooled DOR from Network Meta-analysis

In [15]:
rownames(results.nma.95$dor)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$dor

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,14.443244,46.480593,1.1222626
PCT,8.314756,9.949846,6.8265738
presepsin,11.372290,17.581497,6.5560658
CRP,5.072736,6.628437,3.7886449
CD64,20.560039,65.251860,3.1663166
IL-6,8.232974,15.037211,4.0102368
sTREM-1,6.435750,26.593486,0.4222318
LBP,3.703925,13.032341,0.3917135
SIRS,4.337863,9.251447,1.4698339


### Print the results of pooled Superiority index from Network Meta-analysis

In [16]:
rownames(results.nma.95$sindex)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$sindex

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,1.4664688,5,0.09090909
PCT,3.0241305,9,0.60000000
presepsin,5.8816784,13,1.00000000
CRP,0.4295164,1,0.09090909
CD64,9.3992845,17,0.20000000
IL-6,2.9691787,11,0.14285714
sTREM-1,1.6088342,11,0.05882353
LBP,0.6456096,5,0.05882353
SIRS,0.7163563,3,0.07692308


### Print the results of pooled Superiority index for Sensitivity from Network Meta-analysis

In [17]:
rownames(results.nma.95$sindex_se)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$sindex_se

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,0.1020947,0.3333333,0.05263158
PCT,2.0154658,5.6666667,0.81818182
presepsin,8.6837312,19.0000000,1.85714286
CRP,0.8413185,1.8571429,0.33333333
CD64,10.0677703,19.0000000,0.33333333
IL-6,1.9551572,5.6666667,0.53846154
sTREM-1,0.7176841,5.6666667,0.05263158
LBP,1.5406844,19.0000000,0.05263158
SIRS,6.3767813,19.0000000,0.53846154


### Print the results of pooled Superiority index for Specificity from Network Meta-analysis

In [18]:
rownames(results.nma.95$sindex_sp)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$sindex_sp

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,17.1766824,19.0000000,0.53846154
PCT,1.2779094,3.0000000,0.53846154
presepsin,1.0677171,3.0000000,0.17647059
CRP,0.5152468,1.2222222,0.17647059
CD64,3.0148812,5.6666667,0.17647059
IL-6,1.4688587,5.6666667,0.17647059
sTREM-1,3.5072922,19.0000000,0.05263158
LBP,0.5707948,5.6666667,0.05263158
SIRS,0.1207383,0.3333333,0.05263158


##################################################################################
###                          Univariable Network meta-regression                         ###
##################################################################################

### Define a function to get the results from Univariable Network Meta-regression

In [ ]:
get.nma.1mk<-function(data.ls=NULL,mk.names=NULL,n.study=1,conf=95){
	           data.nma<-data.ls$data.nma
	           func.ub<-NULL
	           func.lb<-NULL
	           if(conf==95){
		                      func.ub<-get.ub.95
		                      func.lb<-get.lb.95
	           } else if(conf==90){
		                             func.ub<-get.ub.90
		                             func.lb<-get.lb.90
	           }
	           data.nma.muse<-data.nma[paste('MU[',1,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.musp<-data.nma[paste('MU[',2,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.rrse<-data.nma[paste('RR[',1,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.rrsp<-data.nma[paste('RR[',2,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.orse<-data.nma[paste('OR[',1,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.orsp<-data.nma[paste('OR[',2,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.dor<-data.nma[paste('DOR[',1:length(data.ls$mk),']',sep='')]
	           data.nma.sindex<-data.nma[paste('S[',1:length(data.ls$mk),']',sep='')]
	           data.nma.sindex_se<-data.nma[paste('S_se[',1:length(data.ls$mk),']',sep='')]
	           data.nma.sindex_sp<-data.nma[paste('S_sp[',1:length(data.ls$mk),']',sep='')]
	           data.nma.beta1se<-data.nma[paste('beta1[1,',1:length(data.ls$mk),']',sep='')]
	           data.nma.beta1sp<-data.nma[paste('beta1[2,',1:length(data.ls$mk),']',sep='')]
	           mu.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.musp,mean))),
	                               ub_sp=array(unlist(lapply(data.nma.musp,func.ub))),
	                               lb_sp=array(unlist(lapply(data.nma.musp,func.lb))),
	                               mean_se=array(unlist(lapply(data.nma.muse,mean))),
	                               ub_se=array(unlist(lapply(data.nma.muse,func.ub))),
	                               lb_se=array(unlist(lapply(data.nma.muse,func.lb))))
	           rr.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.rrsp,mean))),
	                               ub_sp=array(unlist(lapply(data.nma.rrsp,func.ub))),
	                               lb_sp=array(unlist(lapply(data.nma.rrsp,func.lb))),
	                               mean_se=array(unlist(lapply(data.nma.rrse,mean))),
	                               ub_se=array(unlist(lapply(data.nma.rrse,func.ub))),
	                               lb_se=array(unlist(lapply(data.nma.rrse,func.lb))))
	           or.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.orsp,mean))),
	                               ub_sp=array(unlist(lapply(data.nma.orsp,func.ub))),
	                               lb_sp=array(unlist(lapply(data.nma.orsp,func.lb))),
	                               mean_se=array(unlist(lapply(data.nma.orse,mean))),
	                               ub_se=array(unlist(lapply(data.nma.orse,func.ub))),
	                               lb_se=array(unlist(lapply(data.nma.orse,func.lb))))
	           dor.info<-data.frame(mean=array(unlist(lapply(data.nma.dor,mean))),
	                                ub=array(unlist(lapply(data.nma.dor,func.ub))),
	                                lb=array(unlist(lapply(data.nma.dor,func.lb))))
	           sindex.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex,mean))),
	                                   ub=array(unlist(lapply(data.nma.sindex,func.ub))),
	                                   lb=array(unlist(lapply(data.nma.sindex,func.lb))))
	           sindex_se.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_se,mean))),
	                                      ub=array(unlist(lapply(data.nma.sindex_se,func.ub))),
	                                      lb=array(unlist(lapply(data.nma.sindex_se,func.lb))))
	           sindex_sp.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_sp,mean))),
	                                      ub=array(unlist(lapply(data.nma.sindex_sp,func.ub))),
	                                      lb=array(unlist(lapply(data.nma.sindex_sp,func.lb))))
	           beta1.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.beta1sp,mean))),
	                                  ub_sp=array(unlist(lapply(data.nma.beta1sp,func.ub))),
	                                  lb_sp=array(unlist(lapply(data.nma.beta1sp,func.lb))),
	                                  mean_se=array(unlist(lapply(data.nma.beta1se,mean))),
	                                  ub_se=array(unlist(lapply(data.nma.beta1se,func.ub))),
	                                  lb_se=array(unlist(lapply(data.nma.beta1se,func.lb))))
	           results.nma<-list(marker=mk.names[mk.names %in% data.ls$mk],
	                             mu=mu.info,
	                             rr=rr.info,
	                             or=or.info,
	                             dor=dor.info,
	                             sindex=sindex.info,
	                             sindex_se=sindex_se.info,
	                             sindex_sp=sindex_sp.info,
	                             beta1=beta1.info)
	return(results.nma)
}

### Univariable Network Meta-regression (STAN code)

In [ ]:
stan.code.1mk<-'
                data{
                      int N;         //number of comparison (n=121)
                      int Nt;        //number of test (n=7)
                      int Ns;        //number of study (n=107)
                      int TP[N];
                      int Dis[N];    //diseased
                      int TN[N];
                      int NDis[N];   //non-diseased
                      int Study[N];
                      int Test[N];
                      int Covar1[N]; //one additional variable
                    }

                parameters{
                            matrix[2,Nt] logitmu;
                            vector[Ns] nu[2];
                            matrix[Ns,Nt] delta[2];
                            vector<lower=0>[2] tau;
                            vector<lower=0>[2] sigmab; 
                            real<lower=-1,upper=1> rho;
                            matrix[2,Nt] beta1;
                        }

                transformed parameters{
                                        matrix[Ns,2] p_i[Nt];
                                        matrix[2,Nt] MU;
                                        matrix[2,Nt] RR;
                                        matrix[2,Nt] OR;
                                        vector[Nt] DOR;
                                        vector[Nt] S;
                                        vector[Nt] S_se;
                                        vector[Nt] S_sp;
                                        matrix[Nt,Nt] A;
                                        matrix[Nt,Nt] B;
                                        matrix[Nt,Nt] C;
                                        matrix[Nt, Nt] A_se;
                                        matrix[Nt, Nt] B_se;
                                        matrix[Nt, Nt] C_se;
                                        matrix[Nt, Nt] A_sp;
                                        matrix[Nt, Nt] B_sp;
                                        matrix[Nt, Nt] C_sp;

                                        vector<lower=0>[2] tausq;
                                        vector<lower=0>[2] sigmabsq;

                                        matrix[Nt,Nt] sigmasq[2];
                                        matrix[Nt,Nt] rhow[2];

                for (i in 1:Ns){
                     for (j in 1:2){
                          for (k in 1:Nt){
                                           p_i[k][i,j]=inv_logit(logitmu[j,k]+nu[j][i]+delta[j][i,k]+beta1[j,k]*Covar1[i]);
                           }
                      }
                 }

                for (j in 1:2){
                     for (k in 1:Nt){
                                      MU[j,k]=mean(col(p_i[k],j));
                      }
                }
                  
                for (j in 1:2){
                     for (k in 1:Nt){
                                      RR[j,k]=MU[j,k]/MU[j,1]; 
                                      OR[j,k]=(MU[j,k]*(1-MU[j,1]))/(MU[j,1]*(1-MU[j,k]));
                       }
                 }

                for (l in 1:Nt){
                                 DOR[l]=(MU[1,l]*MU[2,l])/((1-MU[1,l])*(1-MU[2,l]));
                     for(m in 1:Nt){
                                     A[l,m]=if_else((MU[1,l]>MU[1,m]) && (MU[2,l]>MU[2,m]),1,0);
                                     B[l,m]=if_else((MU[1,l]<MU[1,m]) && (MU[2,l]<MU[2,m]),1,0);
                                     C[l,m]=if_else((MU[1,l]==MU[1,m]) && (MU[2,l]==MU[2,m]),1,0);
                                     
                                     A_se[l, m] = if_else((MU[1, l] > MU[1, m]), 1, 0);
                                     B_se[l, m] = if_else((MU[1, l] < MU[1, m]), 1, 0);
                                     C_se[l, m] = if_else((MU[1, l] == MU[1, m]), 1, 0);
 
                                     A_sp[l, m] = if_else((MU[2, l] > MU[2, m]), 1, 0);
                                     B_sp[l, m] = if_else((MU[2, l] < MU[2, m]), 1, 0);
                                     C_sp[l, m] = if_else((MU[2, l] == MU[2, m]), 1, 0);
                      }
                        S[l]=(2*sum(row(A,l))+sum(row(C,l)))/(2*sum(row(B,l))+sum(row(C,l)));
                        S_se[l] = (2*sum(row(A_se, l)) + sum(row(C_se, l)))/(2*sum(row(B_se, l)) + sum(row(C_se, l)));
                        S_sp[l] = (2*sum(row(A_sp, l)) + sum(row(C_sp, l)))/(2*sum(row(B_sp, l)) + sum(row(C_sp, l)));
                }
                 tausq=(tau).*(tau);  
                 sigmabsq=(sigmab).*(sigmab);

                for (j in 1:2){
                     for (k in 1:Nt){
                          for (l in 1:Nt){
                                           sigmasq[j][k,l]=(sigmabsq[j]+tausq[j])*((sigmabsq[j]+tausq[j]));
                                           rhow[j][k,l]=sigmabsq[j]/sqrt(sigmasq[j][k,l]);
                            }
                       }
                 }
            }

                model{
                      //Priors
                      for (j in 1:2){
                                      logitmu[j]~normal(0,5);
                                      tau[j]~cauchy(0,2.5);
                                      sigmab[j]~gamma(5,1);
                           for (k in 1:Nt){
                                            beta1[j,k]~normal(0,5);
                           }            
                      }
                       rho~uniform(-1, 1);
                       nu[2]~normal(0,sigmab[2]);

                      for (i in 1:Ns){
                                       nu[1][i]~normal((sigmab[1]/sigmab[2])*rho*nu[2][i],sqrt(sigmabsq[1]*(1-(rho*rho))));
                           for (j in 1:2){
                               for (k in 1:Nt){
                                                delta[j][i,k]~normal(0,tau[j][k]);
                              }
                         }
                      }

                     for (n in 1:N){
                                     TP[n]~binomial(Dis[n],p_i[Test[n]][Study[n],1]);
                                     TN[n]~binomial(NDis[n],p_i[Test[n]][Study[n],2]);
                      }

                    }

                generated quantities{
                                      vector[2*N] loglik;

                                      for (n in 1:N){
                                                      loglik[n]=binomial_lpmf(TN[n]|NDis[n],p_i[Test[n]][Study[n],1]);
                                       }
                                      for (n in (N+1):(2*N)){
                                                              loglik[n]=binomial_lpmf(TN[n-N]|NDis[n-N],p_i[Test[n-N]][Study[n-N],2]);
                                       }
                }
'

### Prepare data for Network Meta-regression

In [ ]:
para.as.ls.1mk<-list(N=nrow(para),
                     Nt=length(unique(para$Test)),
                     Ns=length(unique(para$Study)),
                     TP=para$TP,
                     Dis=para$Dis,
                     TN=para$TN,
                     NDis=para$NDis,
                     Study=as.numeric(para$Study),
                     Test=para$Test,
                     Covar1=data$is.prevalence05)

### Compiling model

In [ ]:
stan.model.1mk<-stan(model_code=stan.code.1mk, 
                     data=para.as.ls.1mk, 
                     iter=1, 
                     warmup=0, 
                     chains=1)

### Running model

In [ ]:
start_time <- Sys.time()
fit.1mk<-stan(fit=stan.model.1mk,
              data=para.as.ls.1mk,
              iter=20000,
              chains=4,
              cores=4,
              control=list(adapt_delta=0.99,
                           stepsize=0.01,
                           max_treedepth=17))
end_time <- Sys.time()
end_time - start_time

### Save the results as .RData and .csv

In [ ]:
filename.simdata.save<-'fit_1mk_simdata_sepsis_isprevalence05.RData'
filename.results.save<-'out_1mk_sepsis_isprevalence05.csv'
results.1mk<-fit.extract(fit.data=fit.1mk,filename.save=filename.simdata.save)
results.1mk.95<-get.nma.1mk(data.ls=list(mk=mk.names[unique(para$Test)],
                                         data.nma=results.1mk),mk.names=mk.names,
                            conf=95)
write.csv(results.1mk.95,filename.results.save)

### Print the results of pooled sensitivity and specificity from Network Meta-regression

In [ ]:
rownames(results.1mk.95$mu)=c('qSOFA', 'PCT', 'presepsin', 'CRP', 'CD64', 'IL-6', 'sTREM-1', 'LBP', 'SIRS', 'SOFA')
results.lmk.95$mu

### Print the results of pooled DOR from Network Meta-regression

In [ ]:
rownames(results.1mk.95$dor)=c('qSOFA', 'PCT', 'presepsin', 'CRP', 'CD64', 'IL-6', 'sTREM-1', 'LBP', 'SIRS', 'SOFA')
results.1mk.95$dor